### The bootstrap

The **bootstrap** is a popular method in statistics, used to estimate the uncertainty associated with an estimator.
In our case, you will have guessed, we want to estimate the quality of a model --- for example, we might want to estimate the MSE of a regression model, as in the previous classes.
It is an additional technique, alternative to cross-validation, to perform model validation.

The main idea is to train the model on a set of $n$ observations, where $n$ is the size of our data set.
But, in this case, where are we going to find data for the test set?
The answer is that we can artificially create a training set as large as the whole data set, by sampling $n$ points from it, *with repetition*.
In this way, there might be points that are taken multiple times, as well as points which are not taken at all.
The set of points which have not been taken will then form our test set.

But can we expect to have enough of these points?
We can easily figure this out.
The chance that a data point will be selected when extracting a sample is $\frac{1}{n}$.
Therefore, the chance of a data point *not* being selected during one extraction is $1 - \frac{1}{n}$.
Because we sample from the data set $n$ times independently, the probability that a point is not selected in any of the $n$ extractions is $\big( 1 - \frac{1}{n} \big)^n$.
If we take the limit $n \to \infty$ of this quantity, we can see that it is $e^{-1} \approx 0.368$.
So, if we have a large enough dataset, roughly 36.8% of the points will not be sampled and the test set size will be approximately 36.8% of the full dataset size.

To make the result robust, we can repeat the bootstrapping procedure $B$ times and, each time, sample the dataset with replacement.
What is the error associated with such a method?
Let $\hat{Y}^{(b)}_i$ be the prediction for the label associated with test data point $X_i$, obtained with the model trained during the $b$-th repetition of the bootstrap procedure.
This means that during the $b$-th iteration point $X_i$ ended up in the test set because it was not sampled for the training set.

To make a concrete example: imagine we are using bootstrap and we are at the first iteration ($b = 1$).
Let $D$ represent the whole data-set: $D = \{ (X_1, Y_1), \ldots, (X_n, Y_n) \}$.
Let $\bar{D}^{(1)}$ be the set of points which were not sampled in the first iteration of the bootstrap.
Therefore, the model has been trained on a set of $n$ points which belong to the set $D \setminus \bar{D}^{(1)}$ (with repetitions).
The set $\bar{D}^{(1)}$ will now be used as our test set.
For each point $(X_i, Y_i) \in \bar{D}^{(1)}$, we can calulate the prediction $\hat{Y}^{(1)}_i$ and then the squared error $\big( Y_i - \hat{Y}^{(1)}_i \big)^2$.

We can repeat the same for the second, third, etc. bootstrap iterations.
At iteration $b$ we'll have a set $\bar{D}^{(b)}$ of points not sampled, which will be our test set.
For each point in that test set, we can calculate the squared error.

When we want to evaluate the MSE of the method we should average the squared error of each point.
But any given point $(X_i, Y_i)$ was in the test set only for some iterations of the bootstrap (when it was not sampled to end up in the training set).
So, for each point $(X_i, Y_i)$ there will be only a subset of bootstrap iterations that will give us the error relative to that point.
Let's call $C_i$ the indices of those bootstrap iterations when $(X_i, Y_i)$ was in the test set.
The average error relative to $(X_i, Y_i)$ is then:

$$\text{SE}_i = \frac{1}{|C_i|} \sum_{b \in C_i} \bigg( \hat{Y}^{(b)}_i - Y_i \bigg)^2 \quad \text{ (or } 0 \text{ if } C_i = \emptyset\text{)}$$

And the error estimation is computed, as in the "regular" MSE calculation, by averaging over all the points in the dataset:

$$\text{Error}_{\,\text{bootstrap}} = \frac{1}{n} \sum_{i=1}^n \text{SE}_i = \frac{1}{n} \sum_{i = 1}^n \frac{1}{|C_i|} \sum_{b \in C_i} \bigg( \hat{Y}^{(b)}_i - Y_i \bigg)^2$$

This estimation is called the **out-of-bag error** because, at each iteration, the points which have not been drawn are called *out-of-bag* samples.

#### Small example

Imagine we have a data-set:
$$D = \{ (X_1, Y_1), (X_2, Y_2), (X_3, Y_3), (X_4, Y_4), (X_5, Y_5) \}$$
and we make three bootstrap iterations.

1. In the first iteration, the training set is:
$$\{ (X_1, Y_1), (X_4, Y_4), (X_1, Y_1), (X_3, Y_3), (X_4, Y_4) \}$$
Where we can see that some point are repeated more than once, while others are missing completely.
The corresponding test set is given by all the points which were not sampled for training:
$$\bar{D}^{(1)} = \{ (X_2, Y_2), (X_5, Y_5) \}$$
The model is trained and then is asked to produce a prediction for each point in the training set.
This means, it will be used to obtain predictions $\hat{Y}_2^{(1)}$ and $\hat{Y}_5^{(1)}$.
2. In the second iteration we have the training set:
$$\{ (X_4, Y_4), (X_2, Y_2), (X_2, Y_2), (X_1, Y_1), (X_2, Y_2) \}$$
and, therfore, the test set is:
$$\bar{D}^{(2)} = \{ (X_3, Y_3), (X_5, Y_5) \}$$
and the predictions we get from the second model are $\hat{Y}_3^{(2)}$ and $\hat{Y}_5^{(2)}$.
3. Finally, in the last iteration we have the training set:
$$\{ (X_5, Y_5), (X_2, Y_2), (X_2, Y_2), (X_3, Y_3), (X_5, Y_5) \}$$
the test set is:
$$\bar{D}^{(3)} = \{ (X_1, Y_1), (X_4, Y_4) \}$$
and the predictions from the third models are $\hat{Y}^{(3)}$ and $\hat{Y}_4^{(3)}$.

We can now calculate what are the sets $C_i$ for each of the 5 points in the data-set.
Because the first point $(X_1, Y_1)$ was in the test set only at the third bootstrap iteration, we have $C_1 = \{3\}$.
Analogously, $(X_2, Y_2)$ was in the test set in the first iteration only, so $C_2 = \{1\}$.
Continuing, $C_3 = \{2\}$, $C_4 = \{5\}$, and $C_5 = \{1,2\}$ because point $(X_5, Y_5)$ was in the test set at both iterations 1 and 2.

So, if we want to know the mean square error relative to point $(X_5, Y_5)$ we should average the two square errors we can obtain with the predictions at the first and second bootstrap iteration:
$$\text{SE}_5 = \frac{1}{|C_5|} \sum_{b \in C_5} \Big( \hat{Y}_5^{(b)} - Y_5 \Big)^2 = \frac{1}{2} \bigg( \Big( \hat{Y}_5^{(1)} - Y_5 \Big)^2 + \Big( \hat{Y}_5^{(2)} - Y_5 \Big)^2 \bigg)$$
If we want to know $\text{SE}_1$, this is going to be even easier, because we only have one iteration which produced an estimation for point $(X_1, Y_1)$, so there is "nothing" to average.
And the same is true, in this small example, also for $\text{SE}_2, \text{SE}_3, \text{SE}_4$.

Finally, the overall estimation is given by averaging $\text{SE}_1, \ldots, \text{SE}_5$.
In our small example, this is:
$$\text{Error}_\text{Bootstrap} = \frac{1}{5} \bigg(
    \big( \hat{Y}_1^{(3)} - Y_1 \big)^2 +
    \big( \hat{Y}_2^{(1)} - Y_1 \big)^2 +
    \big( \hat{Y}_3^{(2)} - Y_1 \big)^2 +
    \big( \hat{Y}_4^{(5)} - Y_1 \big)^2 +
    \frac{1}{2} \big( (\hat{Y}_5^{(1)} - Y_1)^2 + (\hat{Y}_5^{(2)} - Y_1)^2 \big)
\bigg)$$

### Limitations of the Bootstrap

Since each bootstrap sample (training set) has $n$ data points but, on average, only $\sim \, n \cdot 0.632$ of them are distinct, the training set does not adequately represent the distribution of the original data.
This problem is similar to the one we would have if we did validation by splitting the total data-set into a training and test sets with a rough 63%-37% proportion.
Because the model is trained on roughly $n \cdot 0.632$ distinct points over the $n$ available, it will perform slightly worse than a model that can be trained on the whole data set.
For this reason, $\text{Error}_{\,\text{bootstrap}}$ often overestimates the true MSE of the model.

To overcome this problem, a new estimate of the error has been proposed.
It is called the **0.632 method** and it consists of averaging an overestimated and an underestimated value for the true error, to obtain a more accurate prediction.
Concretely, the error is estimated as:

$$\text{Error}_{\,0.632} = 0.632 \cdot \text{Error}_{\,\text{Bootstrap}} + 0.368 \cdot \text{Error}_{\,\text{Training}}$$

Where $\text{Error}_{\,\text{Training}}$ is the training MSE when we use the whole dataset as training set:

$$\text{Error}_{\,\text{Training}} = \frac{1}{n} \sum_{i=1}^n (\hat{Y}_i - Y_i)^2$$

(In the formula above, $\hat{Y}_i$ is the prediction for the $i$-th data point given by the model trained on the whole data set.)
Notice how $\text{Error}_{\,\text{Bootstrap}}$ tends to overestimate the true error, while $\text{Error}_{\,\text{Training}}$ tends to underestimate it.

#### Bonus: the 0.632+ method

If the model is prone to overfit over the entire dataset, $\text{Error}_{\,\text{Train}}$ will be very low.
In that case, the "0.368" component will prevail and $\text{Error}_{\,0.632}$ will also underestimate the true error.
To prevent this situation, a new method called the **0.632+ method** was developed.
The new estimator looks as follows:

$$\text{Error}_{\,0.632+} = w \cdot \text{Error}_{\,\text{Bootstrap}} + (1 - w) \cdot \text{Error}_{\,\text{Training}}$$

Where

$$
\begin{align*}
    w &= \frac{0.632}{1 - 0.368 \cdot R} \\
    \\
    R &= \frac{\text{Error}_{\,\text{Bootstrap}} - \text{Error}_{\,\text{Training}}}{\gamma - \text{Error}_{\,\text{Training}}} \\
    \\
    \gamma &= \frac{1}{n^2} \sum_{i=1}{n} \sum_{j=1}{n} (\hat{Y}_j - Y_i)^2 = \text{"no-information error rate"}
\end{align*}
$$

### Example

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
sns.set_style("darkgrid")
matplotlib.rcParams['figure.figsize'] = (14, 10)

In [2]:
# Data preparation, as we have done before.
d = pd.read_csv('../data/auto-mpg.csv')
label_column = 'mpg'
feature_columns = [c for c in d.columns if c != label_column]
X = d[feature_columns].values
y = d[label_column].values

In [3]:
# Using the Quadratic regression + LASSO model from previous classes
from sklearn.linear_model import Lasso
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.pipeline import make_pipeline

model = make_pipeline(
    StandardScaler(),
    PolynomialFeatures(degree = 2),
    Lasso(alpha = 0.0316, max_iter = 10000))

In [14]:
from sklearn.utils import resample
import numpy as np

# Number of bootstrap iterations; the larger, the better.
bootstrap_iterations = 1000

# Indices of all points in the dataset.
dataset_indices = range(len(X))

# This list of lists will contain the square errors:
# sq_errors[12][5], for example, will contain the square error
# between the true label of point number 12 ans the prediction
# made at the 5-th bootstrap iteration where point number 12
# ended up in the test set.
sq_errors = [list() for _ in dataset_indices]

for b in range(bootstrap_iterations):
    # Indices of the points which will form the training set.
    # It will likely contain duplicates, because we sample with replacement.
    # It has the same size as the full dataset.
    training_indices = resample(dataset_indices, replace = True, n_samples = len(X), random_state = b)
    
    # Indices of the out-of-bag points: those points not sampled
    # for the training set. It contains roughly 36.8% of the points.
    test_indices = [i for i in dataset_indices if i not in training_indices]
    
    # We take training and test data according to the above indices.
    X_train, y_train = X[training_indices], y[training_indices]
    X_test, y_test = X[test_indices], y[test_indices]
    
    # We train the model on the training data...
    model.fit(X_train, y_train)
    # ...and calculate predictions on the testa data.
    predictions = model.predict(X_test)
    
    # We save the squared errors in `sq_errors`.
    for pred_index, point_index in enumerate(test_indices):
        sq_err = (predictions[pred_index] - y_test[pred_index]) ** 2
        sq_errors[point_index].append(sq_err)
    
# We compute, for each point, its average square error.
# This correspond to the inner sum in the definition of Error_Bootstrap.
sq_errors_by_point = [sum(x) / len(x) if len(x) > 0 else 0 for x in sq_errors]

# We compute the bootstrap estimate of the MSE.
# We called this Error_Bootstrap in the explanation above.
bootstrap_error = np.mean(sq_errors_by_point)

In [15]:
print(f"Model MSE (estimated by bootstrap): {bootstrap_error:.3f}")

Model MSE (estimated by bootstrap): 8.041


We can see that, as expected, the estimate is pessimistic because we used models trained on roughly 63.2% of the data.
We can also easily compute the 0.632-bootstrap error.

In [17]:
from sklearn.metrics import mean_squared_error

model.fit(X, y)
all_training_predictions = model.predict(X)
training_error = mean_squared_error(y, all_training_predictions)
bootstrap_632_error = 0.632 * bootstrap_error + 0.368 * training_error

In [18]:
print(f"Model MSE (estimated by the 0.632-bootstrap method): {bootstrap_632_error:.3f}")

Model MSE (estimated by the 0.632-bootstrap method): 7.532


Given the estimates we have seen in previous classes, we can perhaps convince ourselves that this is a better estimation of the true MSE of our Quadratic regression + LASSO model.